# Using Geopy to Get Hospital Coordinatees
### By Brian Minie (bminie7@gmail.com)

In this notebook, I'll demonstrate how to use Geopy to get a hospitals latitude and longitude. Due to the long runtime of this notebook (we are looking up >5000 hospitals), we will write the results to a CSV file so they can be used in other steps/notebooks.

In the future, we could envision these steps as part of a pipeline where the data is pulled down and refreshed on a regular basis.

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Photon

In [2]:
hospital_general_info_df = pd.read_csv("data/Hospital_General_Info.csv")
hospital_general_info_df["full_address"] = hospital_general_info_df.address + " " + hospital_general_info_df.city + "," + hospital_general_info_df.state + " " + hospital_general_info_df.zip_code.astype(str)
subset_df = hospital_general_info_df[["facility_id", "facility_name", "address", "city", "state", "zip_code", "full_address"]]

In [3]:
locator = Photon(user_agent="myGeocoder")
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
subset_df['location'] = subset_df['facility_name'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
subset_df['point'] = subset_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('ORLANDO HEALTH-HEALTH CENTRAL HOSPITAL',), **{}).
Traceback (most recent call last):
  File "C:\Users\bmini\anaconda3\envs\jupyter\lib\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\bmini\anaconda3\envs\jupyter\lib\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\bmini\anaconda3\envs\jupyter\lib\http\client.py", line 1369, in getresponse
    response.begin()
  File "C:\Users\bmini\anaconda3\envs\jupyter\lib\http\client.py", line 310, in begin
    version, status, reason = self._read_status()
  File "C:\Users\bmini\anaconda3\envs\jupyter\lib\http\client.py", line 271, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\bmini\anaconda3\envs\jupyter\lib\socket.py", line 589, in readinto
    return 

In [4]:
geo_df = pd.DataFrame(subset_df['point'].tolist(), index=subset_df.facility_id, columns=['latitude', 'longitude', 'altitude'])
subset_df.set_index("facility_id", inplace=True)
complete = pd.merge(subset_df, geo_df, left_index=True, right_index=True)
complete.to_csv("data/hospital_coordinates.csv")